In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv
/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_deposit_1.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_applprev_2.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_static_cb_0.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_static_0_0.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_credit_bureau_a_1_3.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_credit_bureau_a_1_2.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_tax_registry_b_1.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_static_0_2.parquet
/kaggle/input/home-credit-credit-risk-model-st

## 資料探索與前處理

### 資料概覽
##### 列出主要的數據特徵及統計摘要

In [2]:
# 訓練資料的目錄
projDir = "/kaggle/input/home-credit-credit-risk-model-stability/"
trainDir = os.path.join(projDir, "csv_files/train/")

# 列出目錄下所有文件
files = os.listdir(trainDir)

# 依檔名排序
files = sorted(files)

# 印出文件名稱
for file in files:
    print(file)
    
# 計算文件數量
num_files = len(files)
print("目錄下有", num_files, "個訓練資料文件")

del num_files

train_applprev_1_0.csv
train_applprev_1_1.csv
train_applprev_2.csv
train_base.csv
train_credit_bureau_a_1_0.csv
train_credit_bureau_a_1_1.csv
train_credit_bureau_a_1_2.csv
train_credit_bureau_a_1_3.csv
train_credit_bureau_a_2_0.csv
train_credit_bureau_a_2_1.csv
train_credit_bureau_a_2_10.csv
train_credit_bureau_a_2_2.csv
train_credit_bureau_a_2_3.csv
train_credit_bureau_a_2_4.csv
train_credit_bureau_a_2_5.csv
train_credit_bureau_a_2_6.csv
train_credit_bureau_a_2_7.csv
train_credit_bureau_a_2_8.csv
train_credit_bureau_a_2_9.csv
train_credit_bureau_b_1.csv
train_credit_bureau_b_2.csv
train_debitcard_1.csv
train_deposit_1.csv
train_other_1.csv
train_person_1.csv
train_person_2.csv
train_static_0_0.csv
train_static_0_1.csv
train_static_cb_0.csv
train_tax_registry_a_1.csv
train_tax_registry_b_1.csv
train_tax_registry_c_1.csv
目錄下有 32 個訓練資料文件


In [3]:
# 遍歷每個文件，僅顯示前幾行資料概況
for file in files:
    # 組合完整的檔案路徑
    file_path = os.path.join(trainDir, file)
    
    # 讀取文件的前幾行
    df_head = pd.read_csv(file_path, nrows=3)
    
    print("文件名稱:", file)
    print("前3行資料:")
    display(df_head)
    print("\n")
    
del files

文件名稱: train_applprev_1_0.csv
前3行資料:


,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,2,0.0,640.2,NaN,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,24.0
1,2,0.0,1682.4,NaN,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,...,1,NaN,12.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,12.0
2,3,0.0,6140.0,NaN,NaN,P94_109_143,NaN,2019-01-07,NaN,0.0,...,0,NaN,12.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,12.0




文件名稱: train_applprev_1_1.csv
前3行資料:


,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,40704,0.0,7204.6000,NaN,NaN,P94_109_143,NaN,2018-11-20,NaN,0.0,...,0,NaN,12.0,P46_145_78,a55475b1,P198_131_9,P94_109_143,NaN,D,12.0
1,40734,0.0,3870.2000,NaN,NaN,P94_109_143,NaN,2019-12-26,NaN,0.0,...,0,NaN,18.0,P149_40_170,a55475b1,P45_84_106,P94_109_143,NaN,D,18.0
2,40737,0.0,2324.4001,NaN,1.0,a55475b1,0.0,2014-07-17,NaN,0.0,...,0,0.0,18.0,P46_145_78,a55475b1,a55475b1,a55475b1,NaN,D,18.0




文件名稱: train_applprev_2.csv
前3行資料:


...

,case_id,amount_4527230A,name_4527232M,num_group1,recorddate_4527225D
0,28631,711.0,f980a1ea,3,2019-09-13
1,28631,1946.0,f980a1ea,2,2019-09-13
2,28631,2600.0,f980a1ea,1,2019-09-13




文件名稱: train_tax_registry_b_1.csv
前3行資料:


,case_id,amount_4917619A,deductiondate_4917603D,name_4917606M,num_group1
0,49435,6885.0,2019-10-16,6b730375,0
1,49435,6885.0,2019-10-16,6b730375,1
2,49435,6885.0,2019-10-16,6b730375,2




文件名稱: train_tax_registry_c_1.csv
前3行資料:


,case_id,employername_160M,num_group1,pmtamount_36A,processingdate_168D
0,357,c91b12ff,0,1200.0,2019-01-04
1,357,c91b12ff,1,1200.0,2018-11-28
2,357,c91b12ff,2,972.8,2018-11-01


### 資料清理
##### 處理缺失值、重複值、異常值等

### 特徵工程
##### 提取、轉換或建立新特徵以優化模型

In [4]:
import polars as pl

# 讀取訓練數據
train_base = pl.read_csv(trainDir + 'train_base.csv')

static_dfs = [
    pd.read_csv(trainDir + 'train_static_0_0.csv'),
    pd.read_csv(trainDir + 'train_static_0_1.csv')
]
pd_df = pd.concat(static_dfs, ignore_index=True)
del static_dfs
train_static = pl.from_pandas(pd_df)

train_static_cb = pl.read_csv(trainDir + 'train_static_cb_0.csv')
train_person_1 = pl.read_csv(trainDir + 'train_person_1.csv') 
train_credit_bureau_b_2 = pl.read_csv(trainDir + 'train_credit_bureau_b_2.csv') 

# Merge all dataframes using join with how='inner'
train_data = train_base.join(
    train_static, on='case_id', how='inner'
).join(
    train_static_cb, on='case_id', how='inner'
).join(
    train_person_1, on='case_id', how='inner'
).join(
    train_credit_bureau_b_2, on='case_id', how='inner'
)
del train_base
del train_static
del train_static_cb
del train_person_1
del train_credit_bureau_b_2

# 將目標變量和特徵變量分開
X = train_data.drop(columns=['target'])
y = train_data['target']
del train_data

print(X.dtypes)

/tmp/ipykernel_18/450220401.py:7: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(trainDir + 'train_static_0_0.csv'),
/tmp/ipykernel_18/450220401.py:8: DtypeWarning: Columns (20,45,46,56,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(trainDir + 'train_static_0_1.csv')


[Int64, String, Int64, Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, String, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, String, Float64, Float64, String, String, String, Float64, Float64, Float64, String, Float64, String, Float64, Boolean, Boolean, String, String, Float64, Float64, String, Float64, Float64, Boolean, Boolean, Boolean, String, String, String, String, Float64, String, String, String, Float64, Float64, Float64, String, String, Float64, String, String, String, String, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float6

/tmp/ipykernel_18/450220401.py:35: DeprecationWarning: named `columns` param is deprecated; use positional `*args` instead.
  X = train_data.drop(columns=['target'])


##### 看target值的分布

In [5]:
y.value_counts()

target,count
i64,u32
1,195543
0,2682664


##### 過取樣: 增加target=1的data

In [6]:
#from imblearn.over_sampling import SMOTE

# 設置合成目標樣本數量
#synthetic_target_count = y.value_counts()[0]

# 初始化 SMOTE
#smote = SMOTE(sampling_strategy={1: synthetic_target_count})

# 使用 SMOTE 生成合成少數類樣本
#X, y = smote.fit_resample(X, y)

# 檢查合成後的資料量
#print("合成後的資料量：", len(X))

# 檢查合成後的目標資料分佈
#unique, counts = np.unique(y, return_counts=True)
#print("合成後的目標資料分佈：", dict(zip(unique, counts)))

In [7]:
X.dtypes

[Int64,
 String,
 Int64,
 Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 String,
 String,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 String,
 Float64,
 Float64,
 String,
 String,
 String,
 Float64,
 Float64,
 Float64,
 String,
 Float64,
 String,
 Float64,
 Boolean,
 Boolean,
 String,
 String,
 Float64,
 Float64,
 String,
 Float64,
 Float64,
 Boolean,
 Boolean,
 Boolean,
 String,
 String,
 String,
 String,
 Float64,
 String,
 String,
 String,
 Float64,
 Float64,
 Float64,
 String,
 String,
 Float64,
 String,
 String,
 String,
 String,
 String,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 String,
 Float64,
 Float64,
 Float64,
 Fl

In [8]:
for col in X.columns:
    if not (X[col].dtype in (pl.Int64, pl.Float64)):
        X = X.drop(col)

In [9]:
X.dtypes

[Int64,
 Int64,
 Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float

In [10]:
from sklearn.model_selection import train_test_split

# 將數據劃分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
del X
del y

## 數據分析方法與模型選擇

### 梯度提升樹（Gradient Boosting Decision Tree，GBDT）

In [11]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

# 取得最後三十萬筆資料(ram不夠，sample不work)
X_train = X_train.tail(300000)
y_train = y_train.tail(300000)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [12]:
# 創建 LightGBM 的訓練資料集
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)
del X_train
del y_train

# 設置參數
params = {
    'boosting_type': 'gbdt',         # 使用梯度提升樹
    'objective': 'binary',           # 二元分類任務
    'metric': 'binary_error',        # 評估指標為二元分類錯誤率
    'num_leaves': 31,                # 每棵樹的最大葉子數量
    'learning_rate': 0.05,           # 學習速率
    'feature_fraction': 0.9,         # 特徵抽樣比例
    'bagging_fraction': 0.8,         # 樣本抽樣比例
    'bagging_freq': 5,               # 樣本抽樣的頻率
    'verbose': 0,                    # 顯示訓練信息
    'early_stopping_rounds': 10
}

In [13]:
# 訓練模型
num_round = 100                      # 迭代輪次
bst = lgb.train(params, lgb_train, num_round, valid_sets=[lgb_train, lgb_test])

In [14]:
# 預測
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# 評估模型
accuracy = accuracy_score(y_test, y_pred_binary)
print("準確率:", accuracy)

# 顯示分類報告
print("分類報告:")
print(classification_report(y_test, y_pred_binary))

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


準確率: 0.9315008286400228
分類報告:


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      1.00      0.96    536211
           1       0.00      0.00      0.00     39431

    accuracy                           0.93    575642
   macro avg       0.47      0.50      0.48    575642
weighted avg       0.87      0.93      0.90    575642



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Submission

In [15]:
submissionDir = os.path.join(projDir, "csv_files/test/")
submission_path = os.path.join(submissionDir, 'test_base.csv')
X_submission = pd.read_csv(submission_path)

X_submission['date_decision'] = pd.to_datetime(X_submission['date_decision'])
X_submission.insert(1, 'year', X_submission['date_decision'].dt.year)
X_submission.insert(2, 'month', X_submission['date_decision'].dt.month)
X_submission.insert(3, 'day', X_submission['date_decision'].dt.day)
X_submission = X_submission.drop('date_decision', axis=1)
X_submission

,case_id,year,month,day,MONTH,WEEK_NUM
0,57543,2021,5,14,202201,100
1,57549,2022,1,17,202201,100
2,57551,2020,11,27,202201,100
3,57552,2020,11,27,202201,100
4,57569,2021,12,20,202201,100
5,57630,2021,3,16,202201,100
6,57631,2022,6,4,202201,100
7,57632,2022,2,5,202201,100
8,57633,2022,1,25,202201,100
9,57634,2021,1,27,202201,100


In [16]:
y_submission_pred = bst.predict(X_submission, num_iteration=bst.best_iteration, predict_disable_shape_check=True)
y_submission_pred_binary = [1 if pred > 0.5 else 0 for pred in y_submission_pred]
y_submission_pred_binary

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [17]:
submission = pd.DataFrame({
    "case_id": X_submission["case_id"].to_numpy(),
    "score": y_submission_pred_binary
}).set_index('case_id')
submission.to_csv("./submission.csv")

In [18]:
pd.read_csv("./submission.csv")

,case_id,score
0,57543,0
1,57549,0
2,57551,0
3,57552,0
4,57569,0
5,57630,0
6,57631,0
7,57632,0
8,57633,0
9,57634,0
